In [106]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [134]:
input_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
# Merging the two data frames for better account on missing values and feature engineering
df = pd.concat([input_df, test_df])
df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)
df = df.reindex_axis(input_df.columns, axis=1)
df.shape
df.columns
#deal with missing values
df['Cabin'].fillna('U0',inplace=True)
fare_median = df['Fare'].dropna().median()
df['Fare'].fillna(fare_median, inplace=True)
embarked_mode = df['Embarked'].dropna().mode().values[0]
df['Embarked'].fillna(embarked_mode, inplace=True)
#df.columns[pd.isnull(df).any()]


#transform to numeric data as sklearn accepts only numeric data
import re

df['CabinLetter'] = df['Cabin'].map(lambda x : re.compile("([a-zA-Z]+)").search(x).group())
df['CabinLetter'] = pd.factorize(df['CabinLetter'])[0]

#dividing into quartiles
df['FareBin'] = pd.factorize(pd.qcut(df['Fare'], 4))[0]
df


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,CabinLetter,FareBin
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,U0,S,0,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,U0,S,0,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,U0,S,0,2
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,U0,Q,0,2
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,2,1
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,U0,S,0,3
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,U0,S,0,2
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,U0,C,0,3


In [135]:
#Derived Variables
df['Names'] = df['Name'].map(lambda x : len(re.split(' ',x)))

# What is each person's title? 
df['Title'] = df['Name'].map(lambda x: re.compile(", (.*?)\.").findall(x)[0])
 
# Group low-occuring, related titles together
df['Title'][df.Title == 'Jonkheer'] = 'Master'
df['Title'][df.Title.isin(['Ms','Mlle'])] = 'Miss'
df['Title'][df.Title == 'Mme'] = 'Mrs'
df['Title'][df.Title.isin(['Capt', 'Don', 'Major', 'Col', 'Sir'])] = 'Sir'
df['Title'][df.Title.isin(['Dona', 'Lady', 'the Countess'])] = 'Lady'



def getTicketPrefix(ticket):
    match = re.compile("([a-zA-Z\.\/]+)").search(ticket)
    if match:
        return match.group()
    else:
        return 'U'
 
def getTicketNumber(ticket):
    match = re.compile("([\d]+$)").search(ticket)
    if match:
        return match.group()
    else:
        return '0'
    
    
df['TicketPrefix'] = df['Ticket'].map( lambda x : getTicketPrefix(x.upper()))
df['TicketPrefix'] = df['TicketPrefix'].map( lambda x: re.sub('[\.?\/?]', '', x) )
df['TicketPrefix'] = df['TicketPrefix'].map( lambda x: re.sub('STON', 'SOTON', x) )



# factorize the prefix to create a numerical categorical variable
df['TicketPrefixId'] = pd.factorize(df['TicketPrefix'])[0]

# extract the ticket number
df['TicketNumber'] = df['Ticket'].map( lambda x: getTicketNumber(x) )

# create a feature for the number of digits in the ticket number
df['TicketNumberDigits'] = df['TicketNumber'].map( lambda x: len(x) ).astype(np.int)

# create a feature for the starting number of the ticket number
df['TicketNumberStart'] = df['TicketNumber'].map( lambda x: x[0:1] ).astype(np.int)

# The prefix and (probably) number themselves aren't useful
df.drop(['TicketPrefix', 'TicketNumber'], axis=1, inplace=True)
#df.columns[pd.isnull(df).any()]
df

/home/makala/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/makala/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/makala/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/makala/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:11: SettingWithCopyWarning: 

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,CabinLetter,FareBin,Names,Title,TicketPrefixId,TicketNumberDigits,TicketNumberStart
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,U0,S,0,0,4,Mr,0,5,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,1,7,Mrs,1,5,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,U0,S,0,2,3,Miss,2,7,3
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,1,7,Mrs,3,6,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,U0,S,0,2,4,Mr,3,6,3
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,U0,Q,0,2,3,Mr,3,6,3
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,2,1,4,Mr,3,5,1
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,U0,S,0,3,4,Master,3,6,3
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,U0,S,0,2,7,Mrs,3,6,3
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,U0,C,0,3,5,Mrs,3,6,2


In [136]:
# fill missing ages

df['Embarked'] = pd.factorize(df['Embarked'])[0]
df['Title'] = pd.factorize(df['Title'])[0]

from sklearn import linear_model

age_df = df[['Age','Pclass', 'SibSp', 'Parch', 'Fare', 'CabinLetter', 'Embarked', 'Title','Names']]
known_age = age_df.loc[df.Age.notnull()]
unknown_age = age_df.loc[df.Age.isnull()]
x = known_age.values[:,1::]
y = known_age.values[:,0]

regr =linear_model.LinearRegression()
regr.fit(x,y)

predicted_ages = regr.predict(unknown_age.values[:,1::])
df.loc[df.Age.isnull(), 'Age'] = predicted_ages
df1 = df
df1

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,CabinLetter,FareBin,Names,Title,TicketPrefixId,TicketNumberDigits,TicketNumberStart
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,U0,0,0,0,4,0,0,5,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C85,1,1,1,7,1,1,5,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,U0,0,0,2,3,2,2,7,3
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,C123,0,1,1,7,1,3,6,1
4,5,0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,U0,0,0,2,4,0,3,6,3
5,6,0,3,"Moran, Mr. James",male,28.312481,0,0,330877,8.4583,U0,2,0,2,3,0,3,6,3
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.000000,0,0,17463,51.8625,E46,0,2,1,4,0,3,5,1
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.000000,3,1,349909,21.0750,U0,0,0,3,4,3,3,6,3
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.000000,0,2,347742,11.1333,U0,0,0,2,7,1,3,6,3
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.000000,1,0,237736,30.0708,U0,1,0,3,5,1,3,6,2


In [151]:
df1.columns[pd.isnull(df).any()]
df1 = df.copy()
df1
scaler = preprocessing.StandardScaler()
df1[['Age','Fare','Pclass','Parch','SibSp','Names','CabinLetter', 'FareBin'  ]] =\
scaler.fit_transform(df1[['Age','Fare','Pclass','Parch','SibSp','Names','CabinLetter', 'FareBin' ]])
df1

numerics = df1.loc[:,['Age','Fare','Pclass','Parch','SibSp','Names','CabinLetter', 'FareBin']]
numerics
for i in range(0, numerics.columns.size-1):
    for j in range(0, numerics.columns.size-1):
        col1 = str(numerics.columns.values[i])
        col2 = str(numerics.columns.values[j])
        # multiply fields together (we allow values to be squared)
        if i <= j:
            name = col1 + "*" + col2
            df1 = pd.concat([df1, pd.Series(numerics.iloc[:,i] * numerics.iloc[:,j], name=name)], axis=1)
        # add fields together
        if i < j:
            name = col1 + "+" + col2
            df1 = pd.concat([df1, pd.Series(numerics.iloc[:,i] + numerics.iloc[:,j], name=name)], axis=1)
        # divide and subtract fields from each other
        if not i == j:
            name = col1 + "/" + col2
            df1 = pd.concat([df1, pd.Series(numerics.iloc[:,i] / numerics.iloc[:,j], name=name)], axis=1)
            name = col1 + "-" + col2
            df1 = pd.concat([df1, pd.Series(numerics.iloc[:,i] - numerics.iloc[:,j], name=name)], axis=1)
len(df1.columns)



/home/makala/anaconda/lib/python2.7/site-packages/pandas/core/index.py:926: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 152 but corresponding boolean dimension is 19
  result = getitem(key)


152

In [185]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import cross_validation

train = df1[df1.Survived.notnull()]
train.columns
train.drop(['PassengerId','Name', 'Ticket', 'Cabin'], axis = 1,inplace = True)
train['Sex'] = pd.factorize(train['Sex'])[0]
x = train.values[:,1::]
y = train.values[:,0]

#forest = RandomForestClassifier(oob_score = True, n_estimators = 10000)
#scores = cross_validation.cross_val_score(forest,x,y,cv=10)
#print scores.mean()

/home/makala/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/makala/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [172]:
forest.fit(x,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10000, n_jobs=1, oob_score=True,
            random_state=None, verbose=0, warm_start=False)

In [177]:
imp = forest.feature_importances_
imp = 100.0 * (imp / imp.max())
important_idx = np.where(imp > 15)[0]
important_idx = [index+1 for index in important_idx]

f = train.columns[important_idx]
f

Index([u'Sex', u'Title', u'Age*Fare', u'Age+Pclass', u'Age+SibSp', u'Age-Names', u'Fare-Age', u'Fare-Pclass', u'Fare+Names', u'Fare+CabinLetter', u'Pclass-Fare', u'Names-Age', u'Names+CabinLetter'], dtype='object')

In [ ]:
train = train[['Survived']+list(f)]
train
x = train.values[:,1::]
y = train.values[:,0]
# for i in range (100, 400, 20)
forest = RandomForestClassifier(oob_score = True, n_estimators = 1000)
scores = cross_validation.cross_val_score(forest,x,y,cv=10)
print scores.mean()
#forest.fit(x,y)

In [193]:
forest.fit(x,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1000, n_jobs=1, oob_score=True, random_state=None,
            verbose=0, warm_start=False)

In [201]:
test = df1[df1.Survived.isnull()]
p_ids = test['PassengerId']
test = test[['Sex', u'Title', u'Age*Fare', u'Age+Pclass', u'Age+SibSp', u'Age-Names', u'Fare-Age', u'Fare-Pclass', u'Fare+Names', u'Fare+CabinLetter', u'Pclass-Fare', u'Names-Age', u'Names+CabinLetter']]
test['Sex'] = pd.factorize(test['Sex'])[0]
test_pred = forest.predict(test)
pred = [int(val) for val in list(test_pred) ]
p_ids.shape

(418,)

In [202]:
survived = np.asanyarray(pred)
ans = pd.DataFrame(data = p_ids)
ans["Survived"] = survived
ans.to_csv("rfmodel_uv.csv",index=False)